In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
from joblib import Parallel, delayed
import joblib
import json

In [2]:
def mapping_function(to_map):
    d = {}
    for elements in to_map:
        if elements not in d:
            d[elements] = len(d) + 1
    return d

In [3]:
def extract_companies(to_companies,parameter):
    if(parameter==1):
        companies = []
        for elements in to_companies:
            companies.append(elements.split()[0])
        return companies
    elif(parameter==2):
        names = []
        for elements in to_companies:
            first_word, second_word = elements.split()[:2]
            names.append(first_word+" "+second_word)
        return names

In [4]:
def write_to_json_file(data):
    with open('data.json', 'w') as fp:
        json.dump(data, fp)

In [5]:
data_set = pd.read_csv('pakwheels.csv')
df = pd.DataFrame(data_set)
df = df.dropna()
df.drop(df[df['Price'] == 'Call for price'].index, inplace = True)
x = df.drop(['Price'],axis=1)
y = df['Price'].values

In [6]:
record = {}
mapped_dictionary = mapping_function(x['Engine Type'])
x['Engine Type']=x['Engine Type'].map(mapped_dictionary)
record['Engine Type'] = mapped_dictionary
mapped_dictionary = mapping_function(x['Transmission'])
x['Transmission']=x['Transmission'].map(mapped_dictionary)
record['Transmission'] = mapped_dictionary
mapped_dictionary = mapping_function(x['Registered City'])
x['Registered City']=x['Registered City'].map(mapped_dictionary)
record['Registered City'] = mapped_dictionary
mapped_dictionary = mapping_function(x['Color'])
x['Color']=x['Color'].map(mapped_dictionary)
record['Color'] = mapped_dictionary
mapped_dictionary = mapping_function(x['Assembly'])
x['Assembly']=x['Assembly'].map(mapped_dictionary)
record['Assembly'] = mapped_dictionary
mapped_dictionary = mapping_function(x['Body Type'])
x['Body Type']=x['Body Type'].map(mapped_dictionary)
record['Body Type'] = mapped_dictionary
x['Capacity'] = df["Capacity"].str.replace("cc","")
x['Capacity'] = x['Capacity'].astype(int)
companies = extract_companies(x['Name'],1)
x['Company'] = companies
mapped_dictionary = mapping_function(x['Company'])
x['Company']=x['Company'].map(mapped_dictionary)
record['Company'] = mapped_dictionary
names =  extract_companies(x['Name'],2)
x['Names'] = names
mapped_dictionary = mapping_function(x['Names'])
x['Names']=x['Names'].map(mapped_dictionary)
record['Names'] = mapped_dictionary
write_to_json_file(record)
x = x.drop(['Name'],axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train,x_test = x_train.to_numpy(),x_test.to_numpy()

In [8]:
params = {
    "n_estimators": 500,
    "max_depth": 5,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(x_train, y_train)
score=reg.score(x_test,y_test)
joblib.dump(reg, 'trained_model.pkl')

['trained_model.pkl']